In [50]:
# calculate dM/M0 time series for every ROI
import os
import sys
import pandas as pd
import subprocess
import glob
import re
import numpy as np

local_dir = './data2'
roi_dir = './rois'


In [2]:
def refine_list(lst):
    result_list = str(lst).split(' ')
    result_list[0] = result_list[0].replace("b'", "")
    result_list = result_list[: len(result_list) - 1] # remove extra \n\n
    result_list = [float(e) for e in result_list]
    
    return(result_list)
    

In [8]:
cols = ['subject_id', 'model', 'scalar','GM','WM','acc','caudate', 'thalamus', 'S1', ]

df = pd.DataFrame(columns=cols)
for root, dirs, files in os.walk(local_dir, topdown=False):
    # skip the uneven sessions for now
    if '4907' in root or '5509' in root or '0000' in root or '7586' in root:
        continue
    if 'sub-' in root and 'hex' not in root and 'baseline' not in root:
        scalar_maps = glob.glob(root+"/**/*acq-glymph*.nii.gz", recursive=True)
        sub_name = root.split('/')[4].split('sub-')[1]
        model = root.split('/')[2]    
        scalar = root.split('/')[3]
            
        if len(scalar_maps) > 0:
            # merge scalar volumes into one time series volume
            scalar_ts = os.path.join(root, 'sub-{}_{}_{}_ts.nii.gz'.format(sub_name, model, scalar))
            scalar_maps.sort()
            print(scalar_ts)
            pattern = "x|(.*?)|_space"
            for s in scalar_maps:
                substring = s.split('full')[1].split('_space')[0]
                print(substring)
            print('')
            
            fslmerge_inputs = ' '.join(scalar_maps)
            cmd1="fslmerge -t {} {}".format(scalar_ts, fslmerge_inputs)
            print(cmd1)
            subprocess.check_output(cmd1, shell=True)
            
            for mask in glob.glob(roi_dir+"/*mask.nii.gz"):
                cmd2="fslmeants --transpose -i '{}' -m '{}'".format(scalar_ts, mask)
                result = refine_list(subprocess.check_output(cmd2, shell=True))
                if 'gm' in mask:
                    gm = result
                elif 'wm' in mask:
                    wm = result
                if 'acc' in mask:
                    acc = result
                elif 'caudate' in mask:
                    caudate = result
                elif 'thal' in mask:
                    thalamus = result
                elif 's1' in mask:
                    s1 = result


            df.loc[len(df.index)] = [sub_name, model, scalar, gm, wm, acc, caudate, thalamus, s1]

df.to_csv('ts_data.csv', index=False)
df.head()

./data2/GQI/iso/sub-MB5330/sub-MB5330_GQI_iso_ts.nii.gz
1501
1504
1514
1517
1527
1530
1553
1556
1606
1609
1619
1621

fslmerge -t ./data2/GQI/iso/sub-MB5330/sub-MB5330_GQI_iso_ts.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1501_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1504_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1514_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1517_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1527_space-MNI_desc-preproc_space-MNI_desc-iso_gqiscalar.nii.gz ./data2/GQI/iso/sub-MB5330/baseline/sub-MB5330_ses-1_acq-glymphDTI3mm61dirfull1530_space-MNI_desc-preproc

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,S1
0,MB5330,GQI,iso,"[0.200636, 0.207349, 0.192243, 0.205165, 0.216...","[0.207202, 0.215115, 0.19871, 0.212884, 0.2242...","[0.208031, 0.219377, 0.20253, 0.218277, 0.2291...","[0.209803, 0.221403, 0.204345, 0.220674, 0.235...","[0.185593, 0.197859, 0.18052, 0.195454, 0.2081...","[0.178121, 0.185507, 0.171495, 0.182166, 0.196..."
1,AG6470,GQI,iso,"[0.244378, 0.236701, 0.251958, 0.24621, 0.2342...","[0.252908, 0.246563, 0.261418, 0.255285, 0.243...","[0.274828, 0.270195, 0.286724, 0.279462, 0.266...","[0.302385, 0.297936, 0.307736, 0.303713, 0.286...","[0.259048, 0.256311, 0.265161, 0.26215, 0.2482...","[0.224549, 0.21504, 0.233589, 0.229021, 0.2164..."
2,AM5508,GQI,iso,"[0.207368, 0.197094, 0.186958, 0.189082, 0.186...","[0.201513, 0.192101, 0.183694, 0.183691, 0.181...","[0.232884, 0.219202, 0.212976, 0.213258, 0.211...","[0.264339, 0.253393, 0.245846, 0.241851, 0.243...","[0.200452, 0.190934, 0.181494, 0.178636, 0.178...","[0.183249, 0.172481, 0.164691, 0.167208, 0.166..."
3,AA5325,GQI,iso,"[0.21421, 0.197221, 0.220012, 0.203542, 0.2038...","[0.22085, 0.20253, 0.226644, 0.209422, 0.20945...","[0.24506, 0.22495, 0.250746, 0.230565, 0.22943...","[0.270753, 0.251591, 0.283037, 0.260538, 0.259...","[0.218984, 0.199904, 0.224873, 0.204398, 0.205...","[0.184795, 0.169679, 0.194407, 0.177466, 0.180..."
4,KS5471,GQI,iso,"[0.18363, 0.186319, 0.196978, 0.184978, 0.1902...","[0.186871, 0.191168, 0.20084, 0.18917, 0.19432...","[0.212126, 0.215157, 0.22886, 0.217496, 0.2204...","[0.213795, 0.217234, 0.225773, 0.212375, 0.213...","[0.17819, 0.186217, 0.189658, 0.180295, 0.1813...","[0.158769, 0.159915, 0.169831, 0.159233, 0.163..."


### Normalize time series data to first volume

In [63]:
def normalize_ts(ts):
    # normalize ts based on the first baseline timepoint
    normalized_ts = []
    baseline = sum(ts[0:6])/6
    for tpoint in ts:
        norm_tpoint = 100*(tpoint - baseline)/tpoint
        normalized_ts.append(norm_tpoint)
    return normalized_ts

In [64]:
ts = [0.200636, 0.207349, 0.192243, 0.205165, 0.216784, 0.202897, 0.219422, 0.219491, 0.222456, 0.221574, 0.215319, 0.227749]
new_ts = normalize_ts(ts)
print(new_ts)

[-1.7658844873302988, 1.5288233847281532, -6.208808643227598, 0.4805887943849873, 5.814543508746023, -0.6318476862644756, 6.946887732314891, 6.976140251764294, 8.21600676088753, 7.850650347062365, 5.173718993679138, 10.349112400054437]


In [65]:
norm_df = df.copy()
for row, row in df.iterrows(): #iterate over rows
    row_list = [row.subject_id, row.model, row.scalar] 
    sub_id = row.subject_id
    for c, value in row.items():
        if type(value) == list:
            norm_ts = normalize_ts(value) 
            row_list.append(norm_ts)
            
    norm_df.loc[len(norm_df.index)] = row_list

norm_df = norm_df.tail(40) # only get the newly created rows
norm_df.reset_index(drop=True)
norm_df.to_csv('ts_data_norm2.csv', index=False)


In [66]:
norm_df

,subject_id,model,scalar,GM,WM,acc,caudate,thalamus,S1
40,MB5330,GQI,iso,"[-1.7658844873302988, 1.5288233847281532, -6.2...","[-2.0295814390466145, 1.72357421224305, -6.389...","[-3.5285606472112194, 1.8258522999220625, -6.3...","[-3.954741670360619, 1.4917744264229982, -6.73...","[-4.241718886667785, 2.22060490888292, -7.1711...","[-2.223114250050993, 1.8469204216911892, -6.17..."
41,AG6470,GQI,iso,"[0.2809854678680736, -2.953233544992787, 3.280...","[-0.0005272009320771424, -2.5739195797152514, ...","[-0.8193003138932903, -2.548036294774755, 3.36...","[0.36664076150162944, -1.1211580115640052, 2.0...","[-0.29756390064132193, -1.368584779168011, 2.0...","[-0.20099547685954128, -4.631851438492058, 3.6..."
42,AM5508,GQI,iso,"[7.238580687473482, 2.4031680314976636, -2.888...","[6.989706205885812, 2.4326612910222605, -2.032...","[6.974430760951099, 1.1680246226463986, -1.721...","[5.743697802190887, 1.6720404010108239, -1.346...","[7.565817918171594, 2.958002939933864, -2.0894...","[7.335556174021868, 1.5505089449465936, -3.106..."
43,AA5325,GQI,iso,"[2.179948026080334, -6.246461245675319, 4.7595...","[2.4229869443815644, -6.4034134860679, 4.91747...","[2.8610680377594635, -5.822923612654655, 5.063...","[1.2153143270804125, -6.308453005075691, 5.502...","[3.173291199357035, -6.0684128381623195, 5.709...","[0.9177737492897552, -7.909051797806439, 5.816..."
44,KS5471,GQI,iso,"[-2.745647951496682, -1.2627983905738716, 4.21...","[-3.115161439352985, -0.7973789197634134, 4.05...","[-3.2470481380563037, -1.7925669782221063, 4.3...","[-1.3429531404694306, 0.2613924769296604, 4.03...","[-2.514076734571721, 1.9048565204394103, 3.684...","[-2.591395885426835, -1.8561944366277852, 4.09..."
45,PER5474,GQI,iso,"[5.105116918502822, 0.7051804221983449, 1.1169...","[4.729935034902603, 0.8957242436448157, 1.1044...","[3.885918933897997, 0.4858272508430643, 2.2956...","[3.2527187032193607, 0.05453893074990509, 2.26...","[3.3870330918309035, 0.010540618200032341, 2.2...","[6.900030421244769, 2.599247395370166, -1.1310..."
46,CM5268,GQI,iso,"[0.7704996112123149, -3.2137390927235057, 1.43...","[1.0930252989308988, -3.1362440255625477, 0.83...","[0.13362064845275193, -3.0873986490296272, -0....","[0.18383307878523594, -4.3284886838272305, 0.3...","[0.36436603875118007, -3.8749000900667947, 0.9...","[-0.5644801427933931, -4.237780454277187, 1.67..."
47,CKA6605,GQI,iso,"[-0.6229387224776248, 0.1473544706133445, -5.5...","[-0.13490736143413276, -0.3387733459023181, -6...","[0.6445417265066826, 0.15273148819013335, -5.3...","[-0.7707385382201032, -0.41068404388485646, -6...","[-0.3509368691584406, -0.4775960590255676, -7....","[0.4738118882811577, -1.0813550306844129, -6.4..."
48,EP5270,GQI,iso,"[2.2546210129299733, -1.257848803105344, -1.59...","[2.1273451048399754, -0.7567227615308894, -1.4...","[1.2663332560334188, -0.503214963871542, -0.86...","[1.2242732377671846, -0.6991232088026181, -1.1...","[1.6564187201607614, 0.1387772796480984, -1.70...","[3.5026200665192477, -1.2361601612692261, -1.5..."
49,MB5330,GQI,md,"[0.25046733774869095, 0.26481873757960295, 0.4...","[0.1124190943956523, 0.048438518749107234, 0.1...","[0.8625293631787972, 0.6332028598736318, 0.030...","[2.5857015935942034, 1.282588831724236, 0.3099...","[1.0935877383991264, -0.8630045682521224, -0.0...","[-0.16230586994452056, 1.1332040361139388, 0.6..."
